# Figures present in the paper:

   - 1- Figures of Section 3
   - 2- Figures of Section 5
   

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=8)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from GPPY.spatial_windows import BallWindow, BoxWindow
from GPPY.point_processes import HomogeneousPoissonPointProcess
from GPPY.gravitational_force import force_k
from GPPY.gravity_point_process import GravityPointProcess
import copy
import numpy as np

# 1- Figures of Section 3: 

## 1.1 Simulation of push

In [ ]:
d = 2
r=1
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 2000
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=1) 
gppy = GravityPointProcess(poisson_pp)
push_pp = gppy.pushed_point_pattern()


In [ ]:
#finding the force on each point
points = poisson_pp.points
nb_points = points.shape[0]
force_x, force_y=[],[]
epsilon = gppy.epsilon_critical
for k in range(0,nb_points):
    #force_ = epsilon*force_k(k=k, x=points[k], point_pattern=copy.deepcopy(poisson_pp))
    force = force_k(k=k, x=points[k], point_pattern=copy.deepcopy(poisson_pp))[0].tolist()
    force_x.append(force[0])
    force_y.append(force[1])


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
simu_window.plot(axis=ax[0], color="grey")
poisson_pp.plot(axis=ax[0], s=0.1)
poisson_pp.plot(axis=ax[1], c="g", s=0.1, label="Poisson")
ax[1].quiver(points[:,0], points[:,1], force_x, force_y, units='xy')
push_pp.plot(axis=ax[2], s=0.1)
simu_window.plot(axis=ax[2], color="grey")
ax[1].legend()
plt.savefig("poisson_and_push_and_force.pdf")
plt.show()

## 1.2 Variance reduction 

In [ ]:
from GPPY.monte_carlo_test_functions import (f_1, f_2, f_3, f_4, support_integrands)
import multiprocessing
from GPPY.monte_carlo_tests_setup import mc_results, mc_results_single_n

In [ ]:
d=3
nb_sample=50
nb_core = 70
nb_point_list=[500]
fct_list = [f_1, f_2, f_3, f_4]
fct_names = ["f_1", "f_2", "f_3", "f_4"]

support_window = support_integrands(d) 
simulation_window = BallWindow(center=[0]*d,
                               radius=np.sqrt(d)) #simulation window
intensity = nb_point_list[0]/support_window.volume

In [ ]:
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp_big = [poisson.generate_point_pattern(window=simulation_window) for _ in range(nb_sample)]

#Variance for poisson
poisson_pp = [p.restrict_to_window(support_window) for p in poisson_pp_big]
mc_poisson = mc_results_single_n(poisson_pp, "MC", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)

# variance for push
## f_1
mean_f1_poisson = mc_poisson["mc_results_f_1"]["m_MC"]
std_f1_poisson = mc_poisson["mc_results_f_1"]["std_MC"]
## f_2
mean_f2_poisson = mc_poisson["mc_results_f_2"]["m_MC"]
std_f2_poisson = mc_poisson["mc_results_f_2"]["std_MC"]
## f_3
mean_f3_poisson = mc_poisson["mc_results_f_3"]["m_MC"]
std_f3_poisson = mc_poisson["mc_results_f_3"]["std_MC"]
## f_4
mean_f4_poisson = mc_poisson["mc_results_f_4"]["m_MC"]
std_f4_poisson = mc_poisson["mc_results_f_4"]["std_MC"]

In [ ]:
from GPPY.gravity_point_process import epsilon_critical
epsilon_0 = epsilon_critical(d, intensity)
print("d=", d, "intensity", intensity, ",Nb samples=", nb_sample, ",Epsilon critical=", epsilon_0)

In [ ]:
epsilons = np.linspace(-3/2*epsilon_0, 2*epsilon_0, 20  ).tolist()
epsilons.append(epsilon_0)
epsilons.sort()
print(epsilons)

In [ ]:

means_f1_push, stds_f1_push, means_f2_push, stds_f2_push= [], [], [], []
means_f3_push, stds_f3_push, means_f4_push, stds_f4_push = [], [], [], [] 
#push

for epsilon in epsilons:
    time_start = time.time() 
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp_big]
        #using F
        push_pp_big = [g.pushed_point_pattern(epsilon=epsilon, core_number=nb_core) for g in gpp_pp]
    push_pp = [g.restrict_to_window(support_window) for g in push_pp_big]
    mc_push = mc_results_single_n(push_pp, "MCP", 
                                 fct_list=fct_list, 
                                 fct_names=fct_names)
    
    # variance for push
    ## f_1
    means_f1_push.append(mc_push["mc_results_f_1"]["m_MCP"])
    stds_f1_push.append(mc_push["mc_results_f_1"]["std_MCP"])
    ## f_2
    means_f2_push.append(mc_push["mc_results_f_2"]["m_MCP"])
    stds_f2_push.append(mc_push["mc_results_f_2"]["std_MCP"])
    ## f_3
    means_f3_push.append(mc_push["mc_results_f_3"]["m_MCP"])
    stds_f3_push.append(mc_push["mc_results_f_3"]["std_MCP"])
    ## f_4
    means_f4_push.append(mc_push["mc_results_f_4"]["m_MCP"])
    stds_f4_push.append(mc_push["mc_results_f_4"]["std_MCP"]) 
        


In [ ]:
x = np.linspace(-3/5,3/5, 140)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_f_1 = f_1(points)
z_f_2 = f_2(points)
z_f_3 = f_3(points)
z_f_4 = f_4(points)
fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1, rasterized=True)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2, rasterized=True)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_3, rasterized=True)
ax.set_title(r"$f_3$")
#ax.view_init(elev=40, azim=30)
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4, rasterized=True)
ax.set_title(r"$f_4$")
#ax.set_rasterization_zorder(0)
ax.view_init(elev=40, azim=30)
plt.savefig("linear_stat_used_for_var_comp.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 4))
#f_1
ax[0].plot(epsilons, stds_f1_push, "k.", label=r"Push" )
ax[0].plot(epsilons, stds_f1_push, "k" )
ax[0].plot(0, std_f1_poisson, "r*", label="Poisson")
ax[0].vlines(epsilon_0, ymin=min(stds_f1_push), ymax=max(stds_f1_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
#ax[0].set_title(r"$f_1$")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma$")
#f_2
ax[1].plot(epsilons, stds_f2_push,  "k.", label=r"Push")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].plot(0, std_f2_poisson, "r*", label="Poisson")
ax[1].vlines(epsilon_0, ymin=min(stds_f2_push), ymax=max(stds_f2_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")
ax[1].legend()
#ax[1].set_title(r"$f_2$")
ax[1].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[2].plot(epsilons, stds_f3_push,  "k.", label=r"Push")
ax[2].plot(epsilons, stds_f3_push,  "k")
ax[2].plot(0, std_f3_poisson, "r*", label="Poisson")
ax[2].vlines(epsilon_0, ymin=min(stds_f3_push), ymax=max(stds_f3_push), 
             colors="grey", linestyles='dashed', label=r"$\epsilon_0$")

ax[2].legend()
#ax[2].set_title(r"$f_3$")
ax[2].set_xlabel(r"$\epsilon$")
#ax[2].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
#f_3
ax[3].plot(epsilons, stds_f4_push,  "k.", label=r"Push")
ax[3].plot(epsilons, stds_f4_push,  "k")
ax[3].plot(0, std_f4_poisson, "r*", label="Poisson")
ax[3].vlines(epsilon_0, ymin=min(stds_f4_push), ymax=max(stds_f4_push), colors="grey", 
             linestyles='dashed', label=r"$\epsilon_0$")

ax[3].legend()
#ax[3].set_title(r"$f_3$")
ax[3].set_xlabel(r"$\epsilon$")
#ax[3].set_ylabel(r"$\sigma(\sum_{i=1}^{N} f(x_i)/N)$")
plt.savefig("variance_linear_stat_3d.pdf")
plt.show()

# 2- Figures of Section 5

In [ ]:
from GPPY.spatial_windows import BallWindow
def sphere_number(p,q, d):
    r = (p-q)/2
    window = BallWindow(center= [0]*d, radius=q + r)
    return window.surface/(2*r**(d-1))
def next_points(a_0, p, q):
    r = (p-q)/2
    x_1 = (q**2-r**2 + 2*q*r)/(q+r)
    return np.array([[x_1, np.sqrt((q+r)**2 - x_1**2)], [x_1, - np.sqrt((q+r)**2 - x_1**2)]])
def next_points_2(a, p, q):
    r = (p-q)/2
    b_ = (4*r**2*a[1]**2 - 2*a[1]*(q+r)**2)
    a_ = a[1]**2 + a[0]**2 
    c_ = (q + r)**4 + 4*r**4 - 4*r**2*(q + r)**2 - a[0]**2*(q+r)**2
    delta = b_**2 - 4*a_*c_
    print(delta)
    x_2 = [(-b_-np.sqrt(delta))/(2*a_), (-b_+np.sqrt(delta))/(2*a_)]
    print(x_2)
    x_1 = [((q+r)**2 - 2*r**2 - x_2[0]*a[1])/a[1], ((q+r)**2 - 2*r**2 - x_2[1]*a[1])/a[1]]
    return np.array([[x_1[0], x_2[0]], [x_1[1], x_2[1]]]) 

In [ ]:
import numpy as np

epsilon = 0.5
x = np.array([3,76])
a_0 = [76, 0]
d=2
origin = [0]*d
norm_x = np.linalg.norm(x)
R=8
r=R/epsilon
p, q = norm_x + r, norm_x - r
sphere_nb = sphere_number(p=p, q=q, d=d)
points_next = next_points(a_0, p, q)
print(np.linalg.norm(points_next, axis=1))


In [ ]:

print("sphere number=", sphere_nb)
window0 = BallWindow(center=origin, radius=norm_x)
window1 = BallWindow(center=origin, radius=R)
window2 = BallWindow(center=origin, radius=norm_x-R/epsilon)
window3 = BallWindow(center=origin, radius=norm_x + R/epsilon)
window4 = BallWindow(center=x, radius=R/epsilon)
window5 = BallWindow(center=[np.sqrt((q+r)**2 - (q/(q+r))**2), q/(q+r)], radius=r)
window6 = BallWindow(center=points_next[0,:], radius=r)
window7 = BallWindow(center=points_next[1,:], radius=r)

In [ ]:
import numpy as np
from GPPY.spatial_windows import BallWindow

theta = np.arcsin(r/norm_x)
alpha = np.linspace(-theta, theta, 100)
u, v = (R+5)*np.cos(alpha), (R+5)*np.sin(alpha)
nb_theta = int(180/(theta*57.2958)/2)
print(nb_theta)

fig, ax = plt.subplots(figsize=(4,4))
window3.plot(axis=ax, color="b", linestyle="--")
window2.plot(axis=ax, color="b", linestyle="--")
ax.plot(0, 0, "k.")
plt.plot(a_0[0], a_0[1], 'r.')
plt.fill_between([norm_x-r,np.cos(theta)*(norm_x+r)], 
                 [np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x+r)], 
                 [-np.sin(theta)*(norm_x-r), -np.sin(theta)*(norm_x+r)], facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x-r))
plt.fill_betweenx(y_, np.sqrt((norm_x -r)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x), np.sin(theta)*(norm_x))
plt.fill_betweenx(y_, np.sqrt((norm_x)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
for i in range(0, nb_theta ):
    a,b= np.cos(2*i*theta)*norm_x, np.sin(2*i*theta)*norm_x
    window = BallWindow(center=[a,b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, b, 'r.')
    window = BallWindow(center=[a,-b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, -b, 'r.')
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [np.sin((2*i+1)*theta)*(norm_x-r), np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [-np.sin((2*i+1)*theta)*(norm_x-r), -np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
c, d = np.cos(2*7*theta)*norm_x, np.sin(2*7*theta)*norm_x
window = BallWindow(center=[c,d ], radius=r)
window.plot(axis=ax, color="k")
plt.plot(c, d, 'r.')
plt.plot([0, c], [0, d], 'grey', linestyle="--")
plt.gca().annotate(r'$\|\mathbf{x}\|_2$', xy=(c/2, d -5), xycoords='data', fontsize=8, rotation=-10)
plt.plot([0,  np.cos(theta)*(norm_x+r)], 
        [0, np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([0, np.cos(theta)*(norm_x+r)], 
        [0, -np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([np.cos(15*theta)*(norm_x-r), np.cos(15*theta)*(norm_x+r)], 
             [np.sin((15)*theta)*(norm_x-r), np.sin((15)*theta)*(norm_x+r)], 'b' )
plt.plot([norm_x-r,norm_x],[0, 0], 'k' )
plt.gca().annotate(r'$f(\mathbf{x})$', xy=(norm_x -15, 3), xycoords='data', fontsize=8)
plt.plot([0,-8],[0, 0], 'k' )
plt.gca().annotate(r'$R$', xy=(-6, -6), xycoords='data', fontsize=7)
window1.plot(axis=ax, color="k")
window0.plot(axis=ax, color="grey", linestyle="-")
plt.plot(u,v, "grey")
plt.gca().annotate(r'$\theta$', xy=(R+7, -2), xycoords='data', fontsize=8)
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.savefig("proof_existance_pic.pdf")
plt.show()
